In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('OlistData') \
.getOrCreate()

25/02/28 09:03:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
hdfs_path = '/home/jupyter/data/Datasets/2/'

In [26]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

In [27]:
# Cache Frequently used Data for Better Performance

orders_df.cache()
customers_df.cache()
order_item_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [28]:
orders_items_joined_df = orders_df.join(order_item_df,'order_id','inner')

In [29]:
orders_items_products_df = orders_items_joined_df.join(products_df,'product_id','inner')

In [30]:
orders_items_products_sellers_df = orders_items_products_df.join(sellers_df,'seller_id','inner')

In [31]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [32]:
# GEolocation Data

full_orders_df = full_orders_df.join(geolocation_df,full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [33]:
full_orders_df = full_orders_df.join(reviews_df,'order_id','left')

In [34]:
full_orders_df = full_orders_df.join(payments_df,'order_id','left')

In [35]:
full_orders_df.cache()

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [36]:
from pyspark.sql.functions import *

In [37]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [38]:
# Total Revenues Per Seller


seller_revenue_df = full_orders_df.groupBy('seller_id').agg(sum('price'))

In [39]:
seller_revenue_df.show(5)

+--------------------+------------------+
|           seller_id|        sum(price)|
+--------------------+------------------+
|0ea22c1cfbdc755f8...|1747345.2000000048|
|8e6cc767478edae94...| 1145757.400000001|
|ff063b022a9a0aab9...|         1860394.0|
|7aa4334be125fcdd2...|        2509294.49|
|602044f2c16190c2c...|  511238.809999999|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
# Total Orders Per Custoemr
# Average Review Score Per Seller
# Most Sold Products ( Top 10 )
# Top Custoemrs By Spending

# Optimized Joins For Data integration

In [50]:
from pyspark.sql.functions import *

In [51]:
orders_items_joined_df = orders_df.join(order_item_df,'order_id','inner')

In [52]:
orders_items_products_df = orders_items_joined_df.join(products_df,'product_id','inner')

In [53]:
orders_items_products_sellers_df = orders_items_products_df.join(broadcast(sellers_df),'seller_id','inner')

In [54]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [55]:
# GEolocation Data

full_orders_df = full_orders_df.join(broadcast(geolocation_df),full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [56]:
full_orders_df = full_orders_df.join(broadcast(reviews_df),'order_id','left')

In [57]:
full_orders_df = full_orders_df.join(payments_df,'order_id','left')

In [58]:
full_orders_df.cache()

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

# Aggregation

In [59]:
# Total Orders Per Customer

customer_order_count_df = full_orders_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'))\
.orderBy(desc('total_orders'))
         
customer_order_count_df.show(5)

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
+--------------------+------------+
only showing top 5 rows



In [60]:
#Average Review Score Per Seller

seller_review_df = full_orders_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy(desc('avg_review_score'))

seller_review_df.show()

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|e94b64dc6979b302a...|             5.0|
|1e47defeeadeca0e9...|             5.0|
|f26d70155c2b6326f...|             5.0|
|a56a8043ebf66e421...|             5.0|
|31e60bf8d103ce479...|             5.0|
|5733d806668f8712c...|             5.0|
|53b0300ca793f9834...|             5.0|
|c747d5b92c7648417...|             5.0|
|32e5635e63cb374eb...|             5.0|
|1d953075c2f0dd990...|             5.0|
|2d20f1cd18725c911...|             5.0|
|761681a821d8275bc...|             5.0|
|528ce32f560edd11a...|             5.0|
|297d5eccd19fa9a83...|             5.0|
|1cd9e0cc1839d5551...|             5.0|
|64c9a1db4e73e19aa...|             5.0|
|934ed5cd1b9c1c738...|             5.0|
|89757206b887aed36...|             5.0|
|b3e2faf2c1004d674...|             5.0|
|3c88ed2e76a224793...|             5.0|
+--------------------+----------------+
only showing top 20 rows



In [61]:
# Top 10 Most Sold Products

top_products_df = full_orders_df.groupBy('product_id')\
.agg(count('order_id').alias('total_sold'))\
.orderBy(desc('total_sold'))\
.limit(10)

top_products_df.show()

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     86740|
|422879e10f4668299...|     81110|
|99a4788cb24856965...|     78775|
|389d119b48cf3043d...|     60248|
|d1c427060a0f73f6b...|     59274|
|368c6c730842d7801...|     58358|
|53759a2ecddad2bb8...|     52654|
|53b36df67ebb7c415...|     52105|
|154e7e31ebfa09220...|     42700|
|3dd2a17168ec895c7...|     40787|
+--------------------+----------+



In [ ]:
# Top 10 Customer By Spending

# Window Function and Ranking

In [62]:
from pyspark.sql.window import Window

In [63]:

# Dense Rank for Sellers Based on Revenue

window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

In [64]:
# Rank Top Selling Products Per seller

top_seller_products_df = full_orders_df.withColumn('rank',rank().over(window_spec)).filter(col('rank')<=5)

top_seller_products_df.select('seller_id','price','rank').show()

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
|001cca7ae9ae17fb1...|199.0|   1|
+--------------------+-----+----+
only showing top 20 rows



In [ ]:

# Dense Rank for Sellers Based on Revenue


# Advance Aggregation and Enrichment

In [65]:
full_orders_df

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [66]:
# Total Revenue & Average Order Value (AOV) per Customer

customer_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spent'),
    round(avg('price'),2).alias('AOV')
)\
.orderBy(desc('total_spent'))


customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [29]:
6662844/6876

969.0

In [69]:
# Seller Performance Metrics ( Revenue, Average Review, Order Count)

seller_performance_df = full_orders_df.groupBy('seller_id') \
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('review_score'),2).alias('avg_review_score'),
    round(stddev('price'),2).alias('price_variability')
)\
.orderBy(desc('total_revenue'))

In [70]:
seller_performance_df.show()

+--------------------+------------+--------------------+----------------+-----------------+
|           seller_id|total_orders|       total_revenue|avg_review_score|price_variability|
+--------------------+------------+--------------------+----------------+-----------------+
|4869f7a5dfa277a7d...|      184587| 3.613871732000001E7|            4.09|           111.65|
|53243585a1d6dc264...|       54514| 3.429159294999997E7|            4.12|           499.65|
|4a3ca9315b744ce9f...|      330661| 3.375957083999998E7|            3.77|            59.37|
|7c67e1448b00f6e96...|      233306|3.2282321789999593E7|            3.42|            50.39|
|fa1c13f2614d7b5c4...|       87686|3.0139386310000043E7|            4.38|            307.7|
|da8622b14eb17ae28...|      264433|2.9857669729999945E7|            3.98|            72.92|
|7e93a43ef30c4f03f...|       50226| 2.631570629999995E7|            4.15|           377.24|
|1025f0e2d44d7041d...|      229587|2.2937518519999865E7|            3.89|       

In [71]:
from pyspark.sql.functions import format_number

seller_performance_df = seller_performance_df.withColumn(
    "total_revenue", format_number("total_revenue", 2)
)
seller_performance_df.show(10,truncate=False)

+--------------------------------+------------+-------------+----------------+-----------------+
|seller_id                       |total_orders|total_revenue|avg_review_score|price_variability|
+--------------------------------+------------+-------------+----------------+-----------------+
|4869f7a5dfa277a7dca6462dcf3b52b2|184587      |36,138,717.32|4.09            |111.65           |
|53243585a1d6dc2643021fd1853d8905|54514       |34,291,592.95|4.12            |499.65           |
|4a3ca9315b744ce9f8e9374361493884|330661      |33,759,570.84|3.77            |59.37            |
|7c67e1448b00f6e969d365cea6b010ab|233306      |32,282,321.79|3.42            |50.39            |
|fa1c13f2614d7b5c4749cbc52fecda94|87686       |30,139,386.31|4.38            |307.7            |
|da8622b14eb17ae2831f4ac5b9dab84a|264433      |29,857,669.73|3.98            |72.92            |
|7e93a43ef30c4f03f38b393420bc753a|50226       |26,315,706.30|4.15            |377.24           |
|1025f0e2d44d7041d6cf58b6550e0

In [72]:
# Product Popularity Metrics

product_metrics_df = full_orders_df.groupBy('product_id')\
.agg(
    count('order_id').alias('total_sales'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_price'),
    round(stddev('price'),2).alias('price_volatility'),\
    collect_set('seller_id').alias('unique_sellers')    
)\
.orderBy(desc('total_sales'))

In [73]:
product_metrics_df.show()

+--------------------+-----------+------------------+---------+----------------+--------------------+
|          product_id|total_sales|     total_revenue|avg_price|price_volatility|      unique_sellers|
+--------------------+-----------+------------------+---------+----------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740| 6164630.300000002|    71.07|            3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.510000017|    54.77|            4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775|        6921762.71|    87.87|            4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|3280533.1300000064|    54.45|            4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274| 8220103.329999997|   138.68|           16.58|[a1043bafd471dff5...|
|368c6c730842d7801...|      58358|3181698.9000000013|    54.52|            4.59|[1f50f920176fa81d...|
|53759a2ecddad2bb8...|      52654| 2893017.500000002|    54.94|            4.52|[1

In [75]:
# Monthly Revenue and Order Count Trend ----> HW

# order_purchase_timestamp ---> month

# total_orders 
# total_revenue
# avg_order_value
# min_order_value
# max_orderValues

In [76]:
# Customer Retention Analysis ( First & Last Order )

customer_retention_df = full_orders_df.groupBy('customer_id')\
.agg(
    first('order_purchase_timestamp').alias('first_order_date'),
    last('order_purchase_timestamp').alias('last_order_date'),
    count('order_id').alias('total_orders'),
    round(avg('price'),2).alias('aov')
)\
.orderBy(desc('total_orders'))

In [77]:
customer_retention_df.show()

+--------------------+-------------------+-------------------+------------+------+
|         customer_id|   first_order_date|    last_order_date|total_orders|   aov|
+--------------------+-------------------+-------------------+------------+------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427| 85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752| 43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|  26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001| 36.59|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876| 969.0|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876| 12.49|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|  59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|  34.9|
|24e7dc2ff8c071263...|2017-11-24 16:16:45|2017-11-24 16:16:45|        6597|  59.2|
|7bb

In [ ]:
# HW - Correct the last_order_date

# Extended Enrichment 

In [79]:
# Order Status Flags

full_orders_df = full_orders_df.withColumn('is_delivered', when(col('order_status')== 'delivered',lit(1)).otherwise(lit(0)))\
.withColumn('is_canceled', when(col('order_status')== 'canceled',lit(1)).otherwise(lit(0)))

In [80]:
full_orders_df.where(full_orders_df['order_status']=='canceled').select('order_status','is_delivered','is_canceled').show(20)

+------------+------------+-----------+
|order_status|is_delivered|is_canceled|
+------------+------------+-----------+
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
+------------+------------+-----------+
only showing top 20 rows



In [81]:
# Order Revenue Calcualtion

full_orders_df = full_orders_df.withColumn('order_revenue',col('price')+col('freight_value'))

In [48]:
full_orders_df.select('price','freight_value','order_revenue').show()

+-----+-------------+------------------+
|price|freight_value|     order_revenue|
+-----+-------------+------------------+
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
| 27.9|         3.81|31.709999999999997|
+-----+-------------+------------------+
only showing top

In [83]:
customer_spending_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- total_orders: long (nullable = false)
 |-- total_spent: double (nullable = true)
 |-- AOV: double (nullable = true)



In [84]:
# Customer Segmentation based on spending

customer_spending_df = customer_spending_df.withColumn(
    'customer_segment',
    when(col('AOV') >=1200,"High-Value")
    .when( (col('AOV')<1200) & (col('AOV') >=700),'Medium_Value')
    .otherwise('Low-Value'))


In [85]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|       total_spent|    AOV|customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_Value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High-Value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|      High-Value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High-Value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low-Value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High-Value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_Value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High-Value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High-Value|
|695476b5848d64ba0...|         687|18205

In [86]:
full_orders_df = full_orders_df.join(customer_spending_df.select('customer_id','customer_segment'),'customer_id',how='left')

In [87]:
full_orders_df.select('customer_id','customer_segment').show()

+--------------------+----------------+
|         customer_id|customer_segment|
+--------------------+----------------+
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
|2de342d6e5905a5a8...|       Low-Value|
+--------------------+----------------+
only showing top 20 rows



In [88]:
full_orders_df.select('order_purchase_timestamp').show()

+------------------------+
|order_purchase_timestamp|
+------------------------+
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
|     2017-07-26 17:38:47|
+------------------------+
only showing top 20 rows



In [89]:
#Hourly Order Distribution
full_orders_df = full_orders_df.withColumn('hour_of_day',expr('hour(order_purchase_timestamp)'))


In [90]:
full_orders_df.select('order_purchase_timestamp','hour_of_day').show()

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
|     2017-07-26 17:38:47|         17|
+------------------------+-----------+
only showing top 20 rows



In [91]:
# Weekday vs Weekend Order

full_orders_df = full_orders_df.withColumn('order_day_type',\
                                           when(dayofweek('order_purchase_timestamp').isin(1,7),lit('Weekend')).otherwise(lit('weekday')))

In [92]:
full_orders_df.select('order_purchase_timestamp','order_day_type').show()

+------------------------+--------------+
|order_purchase_timestamp|order_day_type|
+------------------------+--------------+
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
|     2017-07-26 17:38:47|       weekday|
+------------------------+--------

In [93]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [94]:
# a new column frieght category  based on freight_value --> low, med or high

In [95]:
# Order Volume by Customer State

In [99]:
full_orders_df.rdd.getNumPartitions()


11

In [105]:
!mkdir /home/jupyter/data/Datasets/output

In [106]:
full_orders_df.write.mode('overwrite').parquet('/home/jupyter/data/Datasets/output')

In [108]:
!ls -lrt -h /home/jupyter/data/Datasets/output

total 249M
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00006-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00003-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00002-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00004-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00005-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00007-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Oct 21 15:06 part-00000-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Oct 21 15:06 part-00001-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root root 23M Oct 21 15:06 part-00008-c1cfe003-6c77-4643-a842-d01e8e1eee6b-c000.snappy.parquet
-rw-r--r-- 1 root